In [1]:
import pandas as pd
import numpy as np
import panel as pn
pn.extension('tabulator')
import hvplot.pandas

In [ ]:
import pandas as pd

# Define the path to the Excel file
file_path_sentek = '/Users/yaniksitta/albirro_project/24 KSU TAPS Sentek.xlsx'

# Load the Excel file
excel_file_sentek = pd.ExcelFile(file_path_sentek)

# Initialize an empty list to hold the data from each worksheet
all_sheets_sentek = []

# Iterate over each sheet in the Excel file
for sheet_name_sentek in excel_file_sentek.sheet_names:
    # Read the sheet, skipping the first two rows
    sheet_data_sentek = pd.read_excel(file_path_sentek, sheet_name=sheet_name_sentek, skiprows=2)
    
    # Remove the " C" from the column names if present
    sheet_data_sentek.columns = [col.replace(" C", "")
                                     .replace("Temp Sensor #6 (55 cm)", "Temp. Sensor #6 (55 cm)")
                                 for col in sheet_data_sentek.columns]
    
    # Add a new column with the sheet name (Team Number)
    sheet_data_sentek['Team Number'] = sheet_name_sentek
    
    # Append the sheet data to the list
    all_sheets_sentek.append(sheet_data_sentek)

# Combine all sheets into a single DataFrame
combined_sentek = pd.concat(all_sheets_sentek, ignore_index=True)

# Convert the 'Timestamp' column to datetime format and specify the desired format
combined_sentek['Timestamp'] = pd.to_datetime(combined_sentek['Timestamp'])

# Remove all rows where column V1 value is -1
combined_sentek = combined_sentek[combined_sentek["V1"] != -1]

# Round the 'Timestamp' to the nearest hour
combined_sentek['Timestamp'] = combined_sentek['Timestamp'].dt.round('H')

# Show the combined DataFrame
combined_sentek


In [ ]:
import pandas as pd

# Define the path to the Excel file for Mesonet data
file_path_mesonet = 'Mesonet_Monthly_Colby.xlsx'

# Load the Excel file
excel_file_mesonet = pd.ExcelFile(file_path_mesonet)

# Initialize an empty list to hold the data from each worksheet
all_sheets_mesonet = []

# Iterate over each sheet in the Excel file
for sheet_name_mesonet in excel_file_mesonet.sheet_names:
    # Read the sheet, skipping the first two rows
    sheet_data_mesonet = pd.read_excel(file_path_mesonet, sheet_name=sheet_name_mesonet)
    
    # Add a new column with the sheet name (Month)
    sheet_data_mesonet['Month'] = sheet_name_mesonet
    
    # Append the sheet data to the list
    all_sheets_mesonet.append(sheet_data_mesonet)

# Combine all sheets into a single DataFrame
combined_mesonet = pd.concat(all_sheets_mesonet, ignore_index=True)

# Drop rows 2 and 3 (which correspond to indices 1 and 2)
combined_mesonet = combined_mesonet.drop([0, 1], axis=0)

# Remove duplicate Timestamps
combined_mesonet = combined_mesonet.drop_duplicates(subset='Timestamp')

# Drop 'Colby' and 'Month' columns
combined_mesonet = combined_mesonet.drop(columns=['Station', 'Month'], errors='ignore')

# Convert the Timestamp column to datetime in combined_mesonet
combined_mesonet['Timestamp'] = pd.to_datetime(combined_mesonet['Timestamp'])

# Convert the specified columns to float
float_columns = ['AirTemperature', 'Precipitation', 'WindDirection2m', 'WindSpeed2m',
                 'SoilTemperature5cm', 'RelativeHumidity','SoilTemperature10cm', 'SolarRadiation']

# Use pd.to_numeric to convert these columns to float, with error handling
for col in float_columns:
    combined_mesonet[col] = pd.to_numeric(combined_mesonet[col], errors='coerce')

# Round the Timestamp to the nearest minute in combined_mesonet
combined_mesonet['Timestamp'] = combined_mesonet['Timestamp'].dt.round('1min')

# Format the Timestamp to "Year-Month-Day Hours:Rounded Minutes" in combined_mesonet
combined_mesonet['Timestamp'] = combined_mesonet['Timestamp'].dt.strftime('%Y-%m-%d %H:%M')


# Show the combined DataFrame
combined_mesonet


In [ ]:
combined_sentek['Timestamp'] = pd.to_datetime(combined_sentek['Timestamp'])

# Round the Timestamp to the nearest minute in combined_sentek
combined_sentek['Timestamp'] = combined_sentek['Timestamp'].dt.round('1min')

# Format the Timestamp to "Year-Month-Day Hours:Rounded Minutes" in combined_sentek
combined_sentek['Timestamp'] = combined_sentek['Timestamp'].dt.strftime('%Y-%m-%d %H:%M')

# Merge the two datasets using 'Timestamp' from combined_sentek as the basis (left join)
merged_data_weather = pd.merge(combined_sentek, combined_mesonet, on='Timestamp', how='left')

# Interpolate to fill in the NaN values in combined_mesonet columns
# Interpolation will linearly estimate values between known points for numerical columns
mesonet_columns = combined_mesonet.columns.drop('Timestamp')  # All columns except 'Timestamp'
merged_data_weather[mesonet_columns] = merged_data_weather[mesonet_columns].interpolate(method='linear')

# Show the merged DataFrame with interpolated values
merged_data_weather

## Graph #1

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import panel as pn

# Enable Panel extension
pn.extension()

# Remove duplicate timestamps for the same team from combined_sentek
combined_sentek_cleaned = combined_sentek.sort_values('Timestamp').drop_duplicates(subset=['Timestamp', 'Team Number'], keep='first')

# Remove duplicate timestamps in combined_mesonet
combined_mesonet_cleaned = combined_mesonet.sort_values('Timestamp').drop_duplicates(subset=['Timestamp'], keep='first')

# Merge the Sentek and Mesonet dataframes on the 'Timestamp' column
merged_data_temp = pd.merge(combined_sentek_cleaned, combined_mesonet_cleaned[['Timestamp', 'AirTemperature']], on='Timestamp', how='inner')

# Create a list of unique team numbers
team_numbers_temp = merged_data_temp['Team Number'].unique()

# Function to update the graph based on the selected team
def update_temperature_graph(selected_team):
    # Create an empty figure
    fig_temp = go.Figure()

    # Filter data for the selected team
    filtered_data = merged_data_temp[merged_data_temp['Team Number'] == selected_team]

    # Add traces for Temp. Sensor #1, #2, #3, and AirTemperature for the selected team
    fig_temp.add_trace(go.Scatter(x=filtered_data['Timestamp'], y=filtered_data['AirTemperature'], 
                                  mode='lines', line=dict(color='cadetblue'), 
                                  name=f'Air Temperature - Team {selected_team}', visible=True))    
    fig_temp.add_trace(go.Scatter(x=filtered_data['Timestamp'], y=filtered_data['Temp. Sensor #1 (5 cm)'], 
                                  mode='lines', line=dict(color='blue'), 
                                  name=f'Temp. Sensor #1 (5 cm) - Team {selected_team}', visible=True))

    # Add upper limit line (32 degrees Celsius)
    fig_temp.add_trace(go.Scatter(x=filtered_data['Timestamp'], y=[32] * len(filtered_data),
                                  mode='lines', line=dict(color='#d62728', dash='dot'),
                                  name='Upper Temp. Limit for ideal Growth (32°C)', visible=True))

    # Add lower limit line (15 degrees Celsius)
    fig_temp.add_trace(go.Scatter(x=filtered_data['Timestamp'], y=[15] * len(filtered_data),
                                  mode='lines', line=dict(color='#d62728', dash='dot'),
                                  name='Lower Temp. Limit for ideal Growth (15°C)', visible=True))

    # Update the layout for the entire chart
    fig_temp.update_layout(
        title=f"Team {selected_team} Temperature Sensors and Air Temperature",
        xaxis_title="Timestamp",
        yaxis_title="Temperature (°C)",
        hovermode="x unified",
        legend=dict(
            orientation="h",  # Horizontal legend
            y=-0.3,           # Position the legend below the plot
            x=0.5,
            xanchor='center'
    )
)
    return fig_temp

# Panel widgets
team_selector_temp = pn.widgets.Select(name='Team Selector for Temperature Dashboard', options=team_numbers_temp.tolist(), value=team_numbers_temp[0])

# Panel plot
@pn.depends(team_selector_temp.param.value)
def plot_temperature(selected_team):
    return pn.panel(update_temperature_graph(selected_team))

# Layout the dashboard
dashboard_weather_2 = pn.Column(
    "# Temperature Sensors vs. Air Temperature Dashboard",
    team_selector_temp,
    plot_temperature
)

# Show the dashboard
dashboard_weather_2.servable()


### Graph #2  Mesonet Weather Data


In [ ]:
import pandas as pd
import panel as pn
import datetime

# Convert Timestamp to datetime
combined_mesonet['Timestamp'] = pd.to_datetime(combined_mesonet['Timestamp'])

# Get the minimum and maximum dates from the Timestamp column
min_date = combined_mesonet['Timestamp'].min().date()
max_date = combined_mesonet['Timestamp'].max().date()

# Panel extension
pn.extension()

# Date selector widget for daily and weekly information
date_picker = pn.widgets.DatePicker(name='Select Date', value=min_date, start=min_date, end=max_date, linked=True)

# Buttons to move to next or previous day or week
next_day_button = pn.widgets.Button(name='Next Day', button_type='primary')
next_week_button = pn.widgets.Button(name='Next Week', button_type='primary')
previous_day_button = pn.widgets.Button(name='Previous Day', button_type='primary')
previous_week_button = pn.widgets.Button(name='Previous Week', button_type='primary')

# Function to increment or decrement the selected date by one day or one week
def update_date(event):
    if event.obj.name == 'Next Day':
        new_date = date_picker.value + datetime.timedelta(days=1)
    elif event.obj.name == 'Next Week':
        new_date = date_picker.value + datetime.timedelta(weeks=1)
    elif event.obj.name == 'Previous Day':
        new_date = date_picker.value - datetime.timedelta(days=1)
    elif event.obj.name == 'Previous Week':
        new_date = date_picker.value - datetime.timedelta(weeks=1)

    if min_date <= new_date <= max_date:
        date_picker.value = new_date

# Attach the update function to the buttons
next_day_button.on_click(update_date)
next_week_button.on_click(update_date)
previous_day_button.on_click(update_date)
previous_week_button.on_click(update_date)

# Function to update the daily summary based on selected date
def update_daily_summary(selected_date):
    # Filter data for the selected date
    filtered_data = combined_mesonet[combined_mesonet['Timestamp'].dt.date == selected_date]
    
    if filtered_data.empty:
        return pn.pane.Alert("No data available for the selected date.", alert_type="warning")
    
    # Calculate summary metrics
    avg_temp = filtered_data['AirTemperature'].mean()
    min_temp = filtered_data['AirTemperature'].min()
    max_temp = filtered_data['AirTemperature'].max()
    avg_humidity = filtered_data['RelativeHumidity'].mean()
    total_precipitation = filtered_data['Precipitation'].sum()
    
    # Create a summary panel with right-aligned text
    summary = pn.Column(
        pn.pane.Markdown(f"### Summary for {selected_date}"),
        pn.Row(pn.pane.Markdown("**Average Temperature:**", width=150), pn.pane.Markdown(f"{avg_temp:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Min Temperature:**", width=150), pn.pane.Markdown(f"{min_temp:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Max Temperature:**", width=150), pn.pane.Markdown(f"{max_temp:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Average Humidity:**", width=150), pn.pane.Markdown(f"{avg_humidity:.2f}%", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Total Precipitation:**", width=150), pn.pane.Markdown(f"{total_precipitation:.2f} mm", styles={'text-align': 'right'})),
        css_classes=['summary-panel'],
        styles={'background': '#f0f0f5'},
        margin=(10, 15),
        padding=(10, 15),
    )
    return summary

# Interactive panel to update daily view
@pn.depends(date_picker)
def daily_view(date_picker):
    return update_daily_summary(date_picker)

# Function to update the weekly summary based on selected week ending date
def update_weekly_summary(ending_date):
    start_date = ending_date - datetime.timedelta(days=6)
    
    # Filter data for the selected week
    filtered_data = combined_mesonet[(combined_mesonet['Timestamp'].dt.date >= start_date) &
                                     (combined_mesonet['Timestamp'].dt.date <= ending_date)]
    
    if filtered_data.empty:
        return pn.pane.Alert("No data available for the selected week.", alert_type="warning")
    
    # Calculate summary metrics
    avg_temp = filtered_data['AirTemperature'].mean()
    min_temp = filtered_data['AirTemperature'].min()
    max_temp = filtered_data['AirTemperature'].max()
    avg_humidity = filtered_data['RelativeHumidity'].mean()
    total_precipitation = filtered_data['Precipitation'].sum()
    
    # Create a summary panel with right-aligned text
    summary = pn.Column(
        pn.pane.Markdown(f"### Summary for Week\n### **{start_date}** to **{ending_date}**"),
        pn.Row(pn.pane.Markdown("**Average Temperature:**", width=150), pn.pane.Markdown(f"{avg_temp:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Min Temperature:**", width=150), pn.pane.Markdown(f"{min_temp:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Max Temperature:**", width=150), pn.pane.Markdown(f"{max_temp:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Average Humidity:**", width=150), pn.pane.Markdown(f"{avg_humidity:.2f}%", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Total Precipitation:**", width=150), pn.pane.Markdown(f"{total_precipitation:.2f} mm", styles={'text-align': 'right'})),
        css_classes=['summary-panel'],
        styles={'background': '#f0f0f5'},
        margin=(10, 15),
        padding=(10, 15),
    )
    return summary

# Interactive panel to update weekly view
@pn.depends(date_picker)
def weekly_view(date_picker):
    return update_weekly_summary(date_picker)

# Create the dashboard layout with tabs
dashboard_weather_1 = pn.Tabs(
    ("Daily Information", pn.Column(
        pn.pane.Markdown("# Daily Data Summary", styles={'font-size': '20px', 'font-weight': 'bold'}),
        date_picker,
        pn.Row(previous_day_button, next_day_button),
        daily_view
    )),
    ("Weekly Information", pn.Column(
        pn.pane.Markdown("# Weekly Data Summary", styles={'font-size': '20px', 'font-weight': 'bold'}),
        date_picker,
        pn.Row(previous_week_button, next_week_button),
        weekly_view
    ))
)

# Serve the dashboard
dashboard_weather_1.servable()


In [ ]:
import pandas as pd

# Load the Excel file into a dataframe
file_path = "Forecast_weather_gov.xlsx"
forecast_df = pd.read_excel(file_path, header=None)

# Transpose the dataframe to switch rows to columns
forecast_transposed = forecast_df.transpose()

# Set the first row as the header
forecast_transposed.columns = forecast_transposed.iloc[0]
forecast_transposed = forecast_transposed[1:]

# Convert "Date" column to datetime
forecast_transposed['Date'] = pd.to_datetime(forecast_transposed['Date'])

# Add the hour information to the "Date" column
forecast_transposed['Hour (CDT)'] = forecast_transposed['Hour (CDT)'].astype(int)
forecast_transposed['Datetime'] = forecast_transposed.apply(lambda row: row['Date'] + pd.Timedelta(hours=row['Hour (CDT)']), axis=1)

# Drop the original "Date" and "Hour (CDT)" columns if no longer needed
forecast_transposed.drop(columns=['Date', 'Hour (CDT)'], inplace=True)

# Rename "Datetime" column to "Date" if needed
forecast_transposed.rename(columns={'Datetime': 'Date'}, inplace=True)

# Convert temperature columns from Fahrenheit to Celsius and rename them
def fahrenheit_to_celsius(fahrenheit):
    return round((fahrenheit - 32) * 5.0/9.0, 1)

# Convert and rename relevant columns
if 'Temperature (°F)' in forecast_transposed.columns:
    forecast_transposed['Temperature (°C)'] = forecast_transposed['Temperature (°F)'].astype(float).apply(fahrenheit_to_celsius)
    forecast_transposed.drop(columns=['Temperature (°F)'], inplace=True)

if 'Dewpoint (°F)' in forecast_transposed.columns:
    forecast_transposed['Dewpoint (°C)'] = forecast_transposed['Dewpoint (°F)'].astype(float).apply(fahrenheit_to_celsius)
    forecast_transposed.drop(columns=['Dewpoint (°F)'], inplace=True)

if 'Wind Chill (°F)' in forecast_transposed.columns:
    forecast_transposed['Wind Chill (°C)'] = forecast_transposed['Wind Chill (°F)'].astype(float).apply(fahrenheit_to_celsius)
    forecast_transposed.drop(columns=['Wind Chill (°F)'], inplace=True)

# Display the updated dataframe
forecast_transposed


In [ ]:
import pandas as pd
import panel as pn
import datetime


# Convert Date to datetime
forecast_transposed['Date'] = pd.to_datetime(forecast_transposed['Date'])

# Get the minimum and maximum dates from the Date column
min_date_2 = forecast_transposed['Date'].min().date()
max_date_2 = forecast_transposed['Date'].max().date()

# Panel extension
pn.extension()

# Date selector widgets for daily information (made unchangeable)
date_picker_day_1 = pn.widgets.DatePicker(name='Select Date Day 1', value=min_date_2, start=min_date_2, end=max_date_2, disabled=True)
date_picker_day_2 = pn.widgets.DatePicker(name='Select Date Day 2', value=min_date_2 + datetime.timedelta(days=1), start=min_date_2, end=max_date_2, disabled=True)

# Function to update the daily summary based on selected date
def update_daily_summary_2(selected_date_2):
    # Filter data for the selected date
    filtered_data_2 = forecast_transposed[forecast_transposed['Date'].dt.date == selected_date_2]
    
    if filtered_data_2.empty:
        return pn.pane.Alert("No data available for the selected date.", alert_type="warning")
    
    # Calculate summary metrics
    avg_temp_2 = filtered_data_2['Temperature (°C)'].mean()
    min_temp_2 = filtered_data_2['Temperature (°C)'].min()
    max_temp_2 = filtered_data_2['Temperature (°C)'].max()
    avg_humidity_2 = filtered_data_2['Relative Humidity (%)'].mean()
    total_precipitation_2 = filtered_data_2['Precipitation Potential (%)'].mean()
    
    # Create a summary panel with right-aligned text
    summary_2 = pn.Column(
        pn.pane.Markdown(f"### Summary for {selected_date_2}"),
        pn.Row(pn.pane.Markdown("**Average Temperature:**", width=150), pn.pane.Markdown(f"{avg_temp_2:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Min Temperature:**", width=150), pn.pane.Markdown(f"{min_temp_2:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Max Temperature:**", width=150), pn.pane.Markdown(f"{max_temp_2:.2f} °C", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Average Humidity:**", width=150), pn.pane.Markdown(f"{avg_humidity_2:.2f}%", styles={'text-align': 'right'})),
        pn.Row(pn.pane.Markdown("**Average Precipitation Potential:**", width=150), pn.pane.Markdown(f"{total_precipitation_2:.2f}%", styles={'text-align': 'right'})),
        css_classes=['summary-panel'],
        styles={'background': '#f0f0f5'},
        margin=(10, 15),
        padding=(10, 15),
    )
    return summary_2

# Create daily summaries for the first and second day
summary_day_1 = update_daily_summary_2(min_date_2)
summary_day_2 = update_daily_summary_2(min_date_2 + datetime.timedelta(days=1))

# Create dashboard tabs for Day 1 and Day 2
dashboard_forecast = pn.Tabs(
    ("Day 1", pn.Column(
        pn.pane.Markdown("# Day 1 Forecast", styles={'font-size': '20px', 'font-weight': 'bold'}),
        date_picker_day_1,
        summary_day_1
    )),
    ("Day 2", pn.Column(
        pn.pane.Markdown("# Day 2 Forecast", styles={'font-size': '20px', 'font-weight': 'bold'}),
        date_picker_day_2,
        summary_day_2
    ))
)

# Serve the dashboard
dashboard_forecast.servable()
